In [1]:
import pandas as pd
import numpy as np
from pmdarima.arima import AutoARIMA
import plotly.express as px
import plotly.graph_objects as go
from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error
from datetime import date as dt, timedelta
import yfinance as yf
import pymysql
import warnings
warnings.filterwarnings('ignore')
import datetime as dt
import pandas_datareader as web
from dateutil.relativedelta import *
import yfinance as yf
from yahoofinancials import YahooFinancials
from sqlalchemy import create_engine
import mysql.connector
from pandas.tseries.offsets import BDay

In [2]:
#Getting the df from SQL table
def make_con():

    conn=pymysql.connect(host='localhost',
                         port=int(3306),
                         user='root',
                         passwd='Calculator1',
                         db='Options')
    
    current_date = pd.datetime.today().strftime('%Y-%m-%d') #this can be used instead of attributes above

    df_all =pd.read_sql_query(f'SELECT * FROM options.anamolous;',conn)
    
    max_date = (pd.to_datetime(df_all.Time.max()) - BDay(5)).strftime('%Y-%m-%d')
    df_all = df_all[df_all.Time <=max_date]
    df_all = df_all.reset_index()
    df_all = df_all[['Symbol', 'Time', 'per_change', 'Calls', 'Puts', 'Put/Call','TotalVolume', 'Orient']]
    
    return df_all

In [3]:
# df_all = make_con()
# df_all.shape

In [4]:
# time_list = list(df_all.Time.unique())

In [5]:
def get_yahoo_data():
    
    df_all = make_con()
    time_list = list(df_all.Time.unique())

    no_list = ['2022-06-03']

    final_df = pd.DataFrame()

    for t in time_list:

            df = df_all[df_all.Time == t].reset_index(drop=True)
            stocks = df.Symbol.unique().tolist()

            start = df.Time.min()
            end = (pd.to_datetime(start) + BDay(7)).strftime('%Y-%m-%d')

            new = yf.download(stocks, start, end, auto_adjust=True, progress=True)['Close']
            new = new.T

            if new.shape[1] == 7:
                new = new.iloc[:,:-1]
                print('length of dataframe was 7 and deleted last column')
            elif new.shape[1] == 6:
                print('length of dataframe was 6')

            new = new.reset_index()
            new.columns = ['Symbol','cp', 'first', 'second', 'third', 'fourth', 'fifth']

            merged_df = df.merge(new, how='inner', left_on='Symbol', right_on='Symbol')

            merged_df['d1_%chng'] = (merged_df['first'] - merged_df['cp'])/merged_df['cp']*100
            merged_df['d2_%chng'] = (merged_df['second'] - merged_df['cp'])/merged_df['cp']*100
            merged_df['d3_%chng'] = (merged_df['third'] - merged_df['cp'])/merged_df['cp']*100
            merged_df['d4_%chng'] = (merged_df['fourth'] - merged_df['cp'])/merged_df['cp']*100
            merged_df['d5_%chng'] = (merged_df['fifth'] - merged_df['cp'])/merged_df['cp']*100

            final_df = pd.concat([final_df, merged_df],axis=0)

    return final_df, merged_df, new, df, t

In [6]:
final_df, merged_df, new, df, t = get_yahoo_data()

[*********************100%***********************]  22 of 22 completed
length of dataframe was 7 and deleted last column
[*********************100%***********************]  25 of 25 completed
length of dataframe was 6
[*********************100%***********************]  26 of 26 completed
length of dataframe was 7 and deleted last column
[*********************100%***********************]  11 of 11 completed
length of dataframe was 7 and deleted last column
[*********************100%***********************]  7 of 7 completed
length of dataframe was 7 and deleted last column
[*********************100%***********************]  6 of 6 completed
length of dataframe was 7 and deleted last column
[*********************100%***********************]  6 of 6 completed
length of dataframe was 7 and deleted last column
[*********************100%***********************]  10 of 10 completed
length of dataframe was 7 and deleted last column
[*********************100%***********************]  11 of 11 c

In [8]:
# final_df.to_excel('final_along_clean.xlsx')